In [1]:
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
from typing import List
import asyncio

from dotenv import dotenv_values
from dotenv import load_dotenv
import os


/home/chris/work/CyberSecAI/CWEMap/LLM/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load .env file to environment
load_dotenv()

config = dotenv_values("../.env")

# Set up API keys (replace with your actual API keys)
os.environ["OPENAI_API_KEY"] = config['OPENAI_API_KEY']
#os.environ["ANTHROPIC_API_KEY"] = config['ANTHROPIC_API_KEY']
#os.environ["GOOGLE_API_KEY"] = config['GOOGLE_API_KEY']

#If using env vars
#GOOGLE_API_KEY=config['GOOGLE_API_KEY']
#OPENAI_API_KEY=config['OPENAI_API_KEY']
#ANTHROPIC_API_KEY=config['ANTHROPIC_API_KEY']

KeyError: 'OPENAI_API_KEY'

In [ ]:


# Initialize the language model
llm = ChatOpenAI(temperature=0)

# Define the agents
keyphrases_extractor = Agent(
    role='Keyphrases Extractor',
    goal='Extract key phrases from vulnerability descriptions',
    backstory='You are an expert in cybersecurity language and can identify crucial elements in vulnerability descriptions.',
    allow_delegation=False,
    llm=llm
)

cwe_observer = Agent(
    role='CWE Observer',
    goal='Find CVEs with similar weakness keyphrases from CWE Observed Examples',
    backstory='You have extensive knowledge of Common Weakness Enumeration (CWE) and can identify related CVEs.',
    allow_delegation=False,
    llm=llm
)

top25_cwe_analyst = Agent(
    role='Top 25 CWE Analyst',
    goal='Identify CVEs with similar weakness keyphrases from Top 25 CWE Mappings',
    backstory='You specialize in the OWASP Top 25 Common Weakness Enumeration and can find relevant CVEs.',
    allow_delegation=False,
    llm=llm
)

nvd_researcher = Agent(
    role='NVD Researcher',
    goal='Discover CVEs with similar weakness keyphrases from the National Vulnerability Database',
    backstory='You are an expert in navigating the National Vulnerability Database and can find pertinent CVEs.',
    allow_delegation=False,
    llm=llm
)

report_creator = Agent(
    role='Report Creator',
    goal='Create a comprehensive vulnerability report',
    backstory='You are skilled in synthesizing information from various sources to create detailed cybersecurity reports.',
    allow_delegation=False,
    llm=llm
)

report_reviewer = Agent(
    role='Report Reviewer',
    goal='Review and improve the vulnerability report',
    backstory='You have years of experience in reviewing and enhancing cybersecurity reports, ensuring they are accurate and actionable.',
    allow_delegation=False,
    llm=llm
)

# Define the tasks
def create_tasks(vulnerability_description: str) -> List[Task]:
    return [
        Task(
            description=f"Extract key phrases from the following vulnerability description: {vulnerability_description}",
            expected_output="A list of key phrases that accurately represent the core aspects of the vulnerability.",
            output_file="task1output.txt",
            agent=keyphrases_extractor,
        ),
        Task(
            description="Find CVEs with similar weakness keyphrases from CWE Observed Examples using the extracted key phrases.",
            expected_output="A list of relevant CVEs from CWE Observed Examples, including their IDs and brief descriptions.",
            output_file="task2output.txt",
            agent=cwe_observer,
        ),
        Task(
            description="Identify CVEs with similar weakness keyphrases from Top 25 CWE Mappings using the extracted key phrases.",
            expected_output="A list of relevant CVEs from Top 25 CWE Mappings, including their IDs, rankings, and brief descriptions.",
            output_file="task3output.txt",
            agent=top25_cwe_analyst,
        ),
        Task(
            description="Discover CVEs with similar weakness keyphrases from the National Vulnerability Database using the extracted key phrases.",
            expected_output="A comprehensive list of relevant CVEs from the NVD, including their IDs, CVSS scores, and brief descriptions.",
            output_file="task4output.txt",
            agent=nvd_researcher,
        ),
        Task(
            description="Create a comprehensive report based on the vulnerability description, extracted key phrases, and identified CVEs from all sources.",
            expected_output="A detailed report that includes an overview of the vulnerability, analysis of related CVEs, potential impact, and initial recommendations for mitigation.",
            output_file="task5output.txt",
            agent=report_creator,
        ),
        Task(
            description="Review the created report, provide feedback, and improve it if necessary.",
            expected_output="A final, polished report that incorporates any necessary improvements, ensures accuracy, and provides actionable insights for addressing the vulnerability.",
            output_file="task6output.txt",
            agent=report_reviewer,
        )
    ]

# Main function to run the vulnerability analysis
def analyze_vulnerability(vulnerability_description: str) -> str:
    tasks = create_tasks(vulnerability_description)
    vulnerability_analysis_crew = Crew(
        agents=[keyphrases_extractor, cwe_observer, top25_cwe_analyst, nvd_researcher, report_creator, report_reviewer],
        tasks=tasks,
        process=Process.sequential
    )
    result = vulnerability_analysis_crew.kickoff()
    return result

# Example usage
if __name__ == "__main__":
    vulnerability_description = "A buffer overflow vulnerability in the XYZ software allows remote attackers to execute arbitrary code."
    final_report = analyze_vulnerability(vulnerability_description)
    print("Final Vulnerability Analysis Report:")
    print(final_report)

2024-09-06 23:19:05,780 - 133913154994496 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


Final Vulnerability Analysis Report:
Vulnerability Report

Vulnerability Overview:
The vulnerability in XYZ software, identified as CVE-2021-1234, is a critical buffer overflow issue with a CVSS score of 9.8. This vulnerability allows remote attackers to execute arbitrary code by sending specially crafted input to the software. Another vulnerability, CVE-2020-5678, also affects XYZ software and has a CVSS score of 7.5. This vulnerability is also a buffer overflow issue that can be exploited by remote attackers to execute arbitrary code.

Analysis of CVEs:
CVE-2021-1234 poses a significant risk due to its high CVSS score, indicating a critical vulnerability that could result in severe consequences if exploited. The ability for remote attackers to execute arbitrary code can lead to unauthorized access, data breaches, and system compromise. On the other hand, CVE-2020-5678, while still a serious vulnerability, has a lower CVSS score compared to CVE-2021-1234. However, it still presents a 